In [1]:
#Imports
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')

In [2]:
#Env setup
%matplotlib inline

In [3]:
#Object detection imports
from utils import label_map_util
from utils import visualization_utils as vis_util

In [4]:
# What model to use.
MODEL_NAME = 'infgraph_frcnn_resnet50'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'C:/Users/bcham/Documents/Tensorflow/workspace/Tonga/annotations/label_map.pbtxt'



In [5]:
#Load a (frozen) Tensorflow model into memory
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [6]:
#Loading label map
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

W1119 15:32:22.083756 10516 deprecation_wrapper.py:119] From C:\Users\bcham\Documents\Tensorflow\models\research\object_detection\utils\label_map_util.py:137: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [7]:
#Helper code
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [8]:
#Detection
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 29) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (20, 16)


MAX_NUMBER_OF_BOXES = None
MINIMUM_CONFIDENCE = 0.5

In [9]:
def detect_objects(image_path):
    image = Image.open(image_path)
    image_np = load_image_into_numpy_array(image)
    image_np_expanded = np.expand_dims(image_np, axis=0)

    (boxes, scores, classes, num) = sess.run([detection_boxes, detection_scores, detection_classes, num_detections],
                                             feed_dict={image_tensor: image_np_expanded})

    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        max_boxes_to_draw=MAX_NUMBER_OF_BOXES,
        min_score_thresh=MINIMUM_CONFIDENCE,
        use_normalized_coordinates=True,
        line_thickness=1)

    #save results 
    
    detected_boxes = []
    h = image_height = 1000 #Change accordingly
    w = image_width = 1000 #change accordingly
    #Columns' format 'ymin','xmin','ymax', 'xmax'
    for b, box in enumerate(np.squeeze(boxes)):
        if (np.squeeze(scores)[b] > MINIMUM_CONFIDENCE):
            box[0] = int(box[0] * h)
            box[1] = int(box[1] * w)
            box[2] = int(box[2] * h)
            box[3] = int(box[3] * w)
            detected_boxes.append(box)
    np.savetxt('evaluations/evaldir_frcnn_resnet50/detections/image{}.csv'.format(x), detected_boxes, fmt='%i', delimiter=',')
    
    img = Image.fromarray(image_np)
    path ='detection_results/frcnn_resnet50'
    img.save(os.path.join(path,'image{}'.format(x) +'.png'))
    

In [10]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        x =1   
        for image_path in TEST_IMAGE_PATHS:
            detect_objects(image_path)
            x = x + 1

In [15]:
def compute_iou(groundtruth_box, detection_box):
    g_ymin = int(groundtruth_box[0])
    g_xmin = int(groundtruth_box[1])  
    g_ymax = int(groundtruth_box[2])  
    g_xmax = int(groundtruth_box[3])  
    
    d_ymin = int(detection_box[0])  
    d_xmin = int(detection_box[1]) 
    d_ymax = int(detection_box[2])
    d_xmax = int(detection_box[3])  
    
    xa = max(g_xmin, d_xmin)
    ya = max(g_ymin, d_ymin)
    xb = min(g_xmax, d_xmax)
    yb = min(g_ymax, d_ymax)

    intersection = max(0, xb - xa + 1) * max(0, yb - ya + 1)

    boxAArea = (g_xmax - g_xmin + 1) * (g_ymax - g_ymin + 1)
    boxBArea = (d_xmax - d_xmin + 1) * (d_ymax - d_ymin + 1)

    return intersection / float(boxAArea + boxBArea - intersection)

def process_detections(g_boxes, d_boxes, g_negatives, image_np):
    
    # compute iou for each detection box with groundtruth boxes and get a list having iou greater than 0.5
    matches = []
            
    for i in range(len(d_boxes)):
        for j in range(len(g_boxes)):
            iou = compute_iou(g_boxes[j], d_boxes[i])     
            
            if iou > 0.5:
                matches.append([d_boxes[i],g_boxes[j],iou])      
    
    True_positives = []
    if len(matches) > 0:   
        matches = np.array(matches)
        # sort the matches in descending order by iou values
        matches = matches[matches[:, 2].argsort()[::-1][:len(matches)]] 

        # if a detection box has more than one match, keep the one having highest iou and remove others 
        matches = matches[np.unique(matches[:,0], return_index=True)[1]] 

        # again sort the matches in descending order by iou values
        matches = matches[matches[:, 2].argsort()[::-1][:len(matches)]]

        # if a groundtruth box has more than one match, keep the one having highest iou and remove others
        matches = matches[np.unique(matches[:,1], return_index=True)[1]]
    
        True_positives = matches[:,0].tolist()  # True positives in detection boxes
        True_positives_g_boxes = matches[:,1].tolist()  # true positives in groundtruth
    else:
        True_positives = []
        True_positives_g_boxes = []
        
    TP = len(True_positives)
    
    
   
    False_positives = [] # detection boxes having iou <= 0.5 or no intersection with groundtruth boxes 
       
    for r in range(len(d_boxes)):
        if not d_boxes[r] in True_positives:
            False_positives.append(d_boxes[r])
    
    FP = len(False_positives) 
    
    
    
    #True_positives_g_boxes = matches[:,1].tolist()  # true positives in groundtruth
    
    
    False_negatives = [] # ignored groundtruth boxes
    
    for f in range(len(g_boxes)):
        if not g_boxes[f] in True_positives_g_boxes:
            False_negatives.append(g_boxes[f])
    
    FN = len(False_negatives) 
   
    
    # determining true negatives
    neg_matches = []
    
    for a in range(len(g_negatives)):
        for b in range(len(d_boxes)):
            iou = compute_iou(g_negatives[a], d_boxes[b])     
            
            if iou > 0.5:
                neg_matches.append([g_negatives[a],d_boxes[b],iou])      
    
    True_negatives = []   # True negatives in groundtruth boxes        
    if len(neg_matches) > 0 :   
        neg_matches = np.array(neg_matches)
        neg_matches = neg_matches[neg_matches[:, 2].argsort()[::-1][:len(neg_matches)]] 
        neg_matches = neg_matches[np.unique(neg_matches[:,0], return_index=True)[1]] 
        neg_matches = neg_matches[neg_matches[:, 2].argsort()[::-1][:len(neg_matches)]]
        neg_matches = neg_matches[np.unique(neg_matches[:,1], return_index=True)[1]]

        neg = neg_matches[:,0].tolist()

        for p in range(len(g_negatives)):
            if not g_negatives[p] in neg:
                True_negatives.append(g_negatives[p])
    else:
        True_negatives = g_negatives
    
    TN = len(True_negatives)
    
    if (TP + FP)>0:
        precision = (TP/(TP + FP))*100
    else:
        precision = 0
    
    if (TP + FN)> 0:  
        recall = (TP/(TP + FN))*100
    else: 
        recall = 0
    
    specificity = (TN/(TN + FP))*100
    
    accuracy = ((TP + TN )/(TP + FP + TN + FN))*100
    
    # displaying results (true positives - black, false positives- red, false negatives- cyan, true negatives - blue)
     
    fig = plt.figure(figsize=IMAGE_SIZE)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    fig.add_axes(ax)
    
    for tp in True_positives:
        box1 = int(tp[0])
        box2 = int(tp[1])
        box3 = int(tp[2])
        box4 = int(tp[3])
    
        rect1 = plt.Rectangle((box2, box1),box4-box2, box3-box1, linewidth=3, edgecolor='k', facecolor='none')
        ax.add_patch(rect1)
        
    for fp in False_positives:
        box1 = int(fp[0])
        box2 = int(fp[1])
        box3 = int(fp[2])
        box4 = int(fp[3])
    
        rect2 = plt.Rectangle((box2, box1),box4-box2, box3-box1, linewidth=3, edgecolor='r', facecolor='none')
        ax.add_patch(rect2)
        
    for fn in False_negatives:
        box1 = int(fn[0])
        box2 = int(fn[1])
        box3 = int(fn[2])
        box4 = int(fn[3])
    
        rect3 = plt.Rectangle((box2, box1),box4-box2, box3-box1, linewidth=3, edgecolor='c', facecolor='none')
        ax.add_patch(rect3)
        
    for tn in True_negatives:
        box1 = int(tn[0])
        box2 = int(tn[1])
        box3 = int(tn[2])
        box4 = int(tn[3])
    
        rect4 = plt.Rectangle((box2, box1),box4-box2, box3-box1, linewidth=3, edgecolor='b', facecolor='none')
        ax.add_patch(rect4)
    
    plt.imshow(image_np)
    plt.draw()
    path ='evaluations/evaldir_frcnn_resnet50'
    plt.savefig(os.path.join(path,'image{}'.format(k+1) +'.png'))
    plt.close()
    
    return [TP,FP,FN,TN,precision,recall,specificity,accuracy]

In [16]:
import csv
n = 28 # number of test images
results = []
for k in range(n):
    with open('evaluations/evaldir_frcnn_resnet50/detections/image'+ str(k+1)+ '.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        d_boxes =[]
        for row in reader:
            d_boxes.append(row)
        
    with open('test_images/groundtruths/image'+ str(k+1)+ '.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        g_boxes =[]
        for row in reader:
            g_boxes.append(row)
    
    with open('test_images/negatives/image'+ str(k+1)+ '.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        g_negatives =[]
        for row in reader:
            g_negatives.append(row)
    
    test_image = 'test_images/image'+ str(k+1)+ '.jpg'
    image = Image.open(test_image)
    image_np = load_image_into_numpy_array(image)
    
    results.append(process_detections(g_boxes, d_boxes, g_negatives, image_np))
    d_boxes.clear()
    g_boxes.clear()
    g_negatives.clear()
    

with open('evaluations/evaldir_frcnn_resnet50/confusion_matrix.csv','w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['TP', 'FP', 'FN', 'TN', 'Precision', 'Recall', 'Specificity', 'Accuracy'])
    writer.writerows(results)    
    